Company:
Airbnb
◆
Difficulty:
Medium
[Question Link](https://www.interviewmaster.ai/question/stays-listing-earnings-insights)

As a Product Analyst on the Airbnb Stays team, you are investigating how listing amenities and pricing strategies impact hosts' supplemental income. Your focus is on understanding the influence of features like pools or ocean views, and the effect of cleaning fees on pricing. Your goal is to derive insights that will help build a pricing recommendation framework to optimize potential nightly earnings for hosts.

**Question 1 of 3**

What is the overall average nightly price for listings with either a 'pool' or 'ocean view' in July 2024? Consider only listings that have been booked at least once during this period.

In [ ]:
#This question wanna know average nightly price
#by have amenities like pool or ocean view in July2024
SELECT ROUND(AVG(nightly_price),2) AS avg_nightly_price
FROM fct_bookings f
JOIN dim_listings d ON
  f.listing_id = d.listing_id
WHERE (booking_date BETWEEN '2024-07-01' AND '2024-07-31')
  AND ((amenities LIKE '%pool%')
  OR (amenities LIKE '%ocean view%'));

**Question 2 of 3**

For listings with no cleaning fee (ie. NULL values in the 'cleaning_fee' column), what is the average difference in nightly price compared to listings with a cleaning fee in July 2024?

In [ ]:
#This question wanna know number of average difference
#between night with clean and not clean in July 2024
SELECT avg_night_price-avg_night_price_without_clean
FROM(
  SELECT AVG(CASE
      WHEN cleaning_fee IS NULL THEN nightly_price END) AS avg_night_price,
    AVG(CASE
      WHEN cleaning_fee IS NOT NULL THEN nightly_price END
    ) AS avg_night_price_without_clean
  FROM fct_bookings
  WHERE booking_date BETWEEN
    '2024-07-01' AND '2024-07-31');

**Question 3 of 3**

Based on the top 50% of listings by earnings in July 2024, what percentage of these listings have ‘ocean view’ as an amenity? For this analysis, look at bookings that were made in July 2024.

In [ ]:
#This queation wanna know listings top 50% first earning,
#what percentage of listings have 'ocean view' in amenity in July 2024
WITH top50_fist_earning AS(
  SELECT f.listing_id AS listing_id,
    d.amenities AS amenities,
    SUM((f.nightly_price * f.booked_nights)+(CASE
      WHEN f.cleaning_fee IS NOT NULL THEN f.cleaning_fee
      ELSE 0
    END)) AS total_earning_per_list
  FROM fct_bookings f
  JOIN dim_listings d ON
    f.listing_id = d.listing_id
  WHERE (booking_date BETWEEN '2024-07-01' AND '2024-07-31')
  GROUP BY 1,2
),
total_row AS (
  SELECT *,
    percent_rank() OVER (ORDER BY total_earning_per_list) as prn
  FROM top50_fist_earning
),
percentage AS (
  SELECT COUNT(listing_id) AS total_row,
    COUNT(CASE
      WHEN (amenities LIKE '%ocean view%') THEN amenities
    END) AS total_row_with_need_view
  FROM total_row
  WHERE prn <= 0.5
)
SELECT ROUND((total_row_with_need_view*100)::NUMERIC/total_row::NUMERIC, 2) AS percentage_of_view
FROM percentage;